In [ ]:
import pandas as pd
import sys
from cellxgene_mods import CxG_API

sys.path.append('..')
from public_resources.public_resources import parse_data_repo_url,validate_raw_insdc

CxG_API.config()

In [ ]:
collections = CxG_API.get_collections()

In [ ]:
insdc_resources = ['geo','arrex','bioproj','ena']
all_collections = []
for c in collections:
    orgs = ','.join(set([o['label'] for d in c['datasets'] for o in d['organism']]))
    assays = ','.join(set([o['label'] for d in c['datasets'] for o in d['assay']]))
    
    tempc = {
        'collection': c['collection_id'],
        'published_date': c['published_at'],
        'organisms': orgs,
        'assays': assays
    }
    for l in c['links']:
        url = l['link_url']
        resource = parse_data_repo_url(url)
        if resource in insdc_resources:
            raw_formats = validate_raw_insdc(url)
            if raw_formats:
                if l['link_type'] != 'RAW_DATA':
                    print('ERROR:raw but not labeled RAW')
                    print(url)
                    print(c['collection_id'])
            elif not raw_formats and l['link_type'] == 'RAW_DATA':
                print('ERROR:no raw but labeled RAW')
                print(url)
                print(c['collection_id'])
        if l['link_type'] == 'RAW_DATA':
            if resource in tempc:
                tempc[resource].append(url)
            else:
                tempc[resource] = [url]
    all_collections.append(tempc)

df = pd.DataFrame(all_collections)
df

In [ ]:
#need to add a new base url?
df['other'].dropna().tolist()

In [ ]:
insdc_df = df.dropna(subset=insdc_resources, how='all')[['collection','organisms','assays'] + insdc_resources]
insdc_df